In [23]:
from nltk.tokenize import word_tokenize
import pandas as pd
import plotly as plt
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
from sklearn.base import clone, BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.metrics import accuracy_score
import seaborn as sns
import string
import textblob
#import xgboost as xgb
from sklearn.model_selection import KFold , train_test_split
from IPython.display import HTML, display
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import sklearn.metrics
from sklearn.metrics import confusion_matrix, classification_report, r2_score
import statsmodels.api as sm
from sklearn.neighbors import KNeighborsClassifier , KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [4]:
# amazon
amazon = open('sentiment labelled sentences/amazon_cells_labelled.txt').read()

a_labels, a_texts = [], []
for i, line in enumerate(amazon.split('\n')):
    content = line.split('\t')
    if len(content) > 1:
        a_texts.append(content[0])
        a_labels.append(content[1])

df_a = pd.DataFrame()
df_a['label'] = a_labels
df_a['text'] = a_texts
df_a['source'] = 'amazon'

# imdb
imdb = open('sentiment labelled sentences/imdb_labelled.txt').read()

i_labels, i_texts = [], []
for i, line in enumerate(imdb.split('\n')):
    content = line.split('\t')
    if len(content) > 1:
        i_texts.append(content[0])
        i_labels.append(content[1])

df_i = pd.DataFrame()
df_i['label'] = i_labels
df_i['text'] = i_texts
df_i['source'] = 'imdb'

# yelp
yelp = open('sentiment labelled sentences/yelp_labelled.txt').read()

y_labels, y_texts = [], []
for i, line in enumerate(yelp.split('\n')):
    content = line.split('\t')
    if len(content) > 1:
        y_texts.append(content[0])
        y_labels.append(content[1])

df_y = pd.DataFrame()
df_y['label'] = y_labels
df_y['text'] = y_texts
df_y['source'] = 'yelp'

In [5]:
df_a.head()

,label,text,source
0,0,So there is no way for me to plug it in here i...,amazon
1,1,"Good case, Excellent value.",amazon
2,1,Great for the jawbone.,amazon
3,0,Tied to charger for conversations lasting more...,amazon
4,1,The mic is great.,amazon


In [7]:
print(df_a['label'].value_counts()) #amazon 2a

0    500
1    500
Name: label, dtype: int64


In [8]:
print(df_i['label'].value_counts()) #imdb 2a

0    500
1    500
Name: label, dtype: int64


In [9]:
print(df_y['label'].value_counts()) #yelp 2a

0    500
1    500
Name: label, dtype: int64


In [20]:
df = pd.concat([df_a, df_i, df_y], ignore_index=True) #combine to 1 DF
df.label = df.label.astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   3000 non-null   int64 
 1   text    3000 non-null   object
 2   source  3000 non-null   object
dtypes: int64(1), object(2)
memory usage: 70.4+ KB


In [21]:
#content exploration
df['chars'] = df.text.apply(len)
df['words'] = df.text.apply(lambda x: len(x.split()))
df['avg_wlen'] = df['chars'] / df['words']
df['puncs'] = df.text.apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
df['uppers'] = df.text.apply(lambda x: len([word for word in x.split() if word.isupper()]))
df['titles'] = df.text.apply(lambda x: len([word for word in x.split() if word.istitle()]))
df.head()


,label,text,source,chars,words,avg_wlen,puncs,uppers,titles
0,0,So there is no way for me to plug it in here i...,amazon,82,21,3.904762,1,2,2
1,1,"Good case, Excellent value.",amazon,27,4,6.750000,2,0,2
2,1,Great for the jawbone.,amazon,22,4,5.500000,1,0,1
3,0,Tied to charger for conversations lasting more...,amazon,79,11,7.181818,3,1,1
4,1,The mic is great.,amazon,17,4,4.250000,1,0,1


In [22]:
pos_group = {
    'noun' : ['NN','NNS','NNP','NNPS'], # Singular Noun, Plural Noun, Proper Singular Noun, Proper Plural Noun
    'pron' : ['PRP','PRP$','WP','WP$'], # Personal Pronoun, Possessive Pronoun, Personal wh-Pronoun, Possessive wh-Pronoun
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'], # Base Verb, Past Tense, Gerund, Past Participle, 1st or 2nd, 3rd
    'adj' :  ['JJ','JJR','JJS'], # Adjective, Comparative, Superlative
    'adv' : ['RB','RBR','RBS','WRB'] # Adverb, Comparative, Superlative, wh-Adverb
}

# check POS and obtain tag count for input
def pos_counts(x, part):
    count = 0
    try:
        wiki = textblob.TextBlob(x)
        for item in wiki.tags:
            pos = list(item)[1]
            if pos in pos_group[part]:
                count += 1
    except:
        pass
    return count

# count tags in dataset
df['nouns'] = df.text.apply(lambda x: pos_counts(x, 'noun'))
df['verbs'] = df.text.apply(lambda x: pos_counts(x, 'verb'))
df['adjs'] = df.text.apply(lambda x: pos_counts(x, 'adj'))
df['advs'] = df.text.apply(lambda x: pos_counts(x, 'adv'))
df['prons'] = df.text.apply(lambda x: pos_counts(x, 'pron'))
df.head()


,label,text,source,chars,words,avg_wlen,puncs,uppers,titles,nouns,verbs,adjs,advs,prons
0,0,So there is no way for me to plug it in here i...,amazon,82,21,3.904762,1,2,2,3,3,0,2,3
1,1,"Good case, Excellent value.",amazon,27,4,6.750000,2,0,2,3,0,1,0,0
2,1,Great for the jawbone.,amazon,22,4,5.500000,1,0,1,2,0,0,0,0
3,0,Tied to charger for conversations lasting more...,amazon,79,11,7.181818,3,1,1,2,3,2,0,0
4,1,The mic is great.,amazon,17,4,4.250000,1,0,1,1,1,1,0,0


In [27]:
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [29]:
#2b) text pre-processing

i = 0
df['clean_text'] = ''
for row in df.text:
    # add spaces to prevent word merging
    row = row.replace('.', '. ', row.count('.')).replace(',', ', ', row.count(','))
    # split into words
    tokens = word_tokenize(row)
    # convert to lower case
    tokens = [token.lower() for token in tokens]
    # remove punctuation
    table = str.maketrans('', '', string.punctuation)
    words = [token.translate(table) for token in tokens]
    # remove non-alphabetic or numeric tokens
    words = [word for word in words if word.isalnum()]
    # filter stop words
    words = [word for word in words if not word in stop_words]
    #print(words)
    df['clean_text'][i] = ' '.join(words)
    i += 1
df.clean_text = df.source + ' ' + df.clean_text
df.head()

<ipython-input-29-4292d4783b50>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text'][i] = ' '.join(words)


,label,text,source,chars,words,avg_wlen,puncs,uppers,titles,nouns,verbs,adjs,advs,prons,clean_text
0,0,So there is no way for me to plug it in here i...,amazon,82,21,3.904762,1,2,2,3,3,0,2,3,amazon way plug us unless go converter
1,1,"Good case, Excellent value.",amazon,27,4,6.750000,2,0,2,3,0,1,0,0,amazon good case excellent value
2,1,Great for the jawbone.,amazon,22,4,5.500000,1,0,1,2,0,0,0,0,amazon great jawbone
3,0,Tied to charger for conversations lasting more...,amazon,79,11,7.181818,3,1,1,2,3,2,0,0,amazon tied charger conversations lasting 45 m...
4,1,The mic is great.,amazon,17,4,4.250000,1,0,1,1,1,1,0,0,amazon mic great


In [37]:
label = list(df['label'])
data = list(df['clean_text'])

In [38]:
#1C split test/training set
# 1-c Split Training and Testing Set
train_data, train_label = [], []
test_data, test_label = [], []
for f in range(0, 3):
    start = f*1000
    one = 0
    zero = 0
    for i in range(start, start+1000):
        if label[i] == 1:
            one += 1
            if one <= 400:
                train_data.append(data[i])
                train_label.append(1)
            else:
                test_data.append(data[i])
                test_label.append(1)
        else:
            zero += 1
            if zero <= 400:
                train_data.append(data[i])
                train_label.append(0)
            else:
                test_data.append(data[i])
                test_label.append(0)

train_data = np.array(train_data)
test_data = np.array(test_data)
train_label = np.array(train_label)
test_label = np.array(test_label)

In [39]:
#1D
word_dict = dict()
index = 0
for s in train_data:
    for w in s.split():
        if w not in word_dict:
            word_dict[w] = index
            index += 1

# Get training and test features
train_feat, test_feat = [], []
word_count = len(word_dict)
for s in train_data:
    cur = [0] * word_count
    for w in s.split():
        cur[word_dict[w]] += 1
    train_feat.append(cur)
train_feat = np.array(train_feat, dtype=float)
for s in test_data:
    cur = [0] * word_count
    for w in s.split():
        if w in word_dict:
            cur[word_dict[w]] += 1
    test_feat.append(cur)
test_feat = np.array(test_feat, dtype=float)

In [40]:
# 1E Postprocessing Strategy  
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
train_post = transformer.fit_transform(train_feat).toarray()
test_post = transformer.fit_transform(test_feat).toarray()

/usr/lib/python3/dist-packages/sklearn/feature_extraction/text.py:1466: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1
